In [ ]:
import argparse
import time

import torch
from PIL import Image
from torch.autograd import Variable
from torchvision.transforms import ToTensor, ToPILImage

from model import Generator

In [ ]:
UPSCALE_FACTOR = 4
TEST_MODE = True #if opt.test_mode == 'GPU' else False
IMAGE_NAME = '001.jpg'
MODEL_NAME = 'netG_epoch_4_100.pth'


In [ ]:
model = Generator(UPSCALE_FACTOR).eval()
if TEST_MODE:
    model.cuda()
    model.load_state_dict(torch.load('epochs/' + MODEL_NAME))
else:
    model.load_state_dict(torch.load('epochs/' + MODEL_NAME, map_location=lambda storage, loc: storage))

image = Image.open(IMAGE_NAME)


In [ ]:
def getblock(image, width_block, height_block, border):
    width = len(image[0][0])
    height = len(image[0])
    if width_block > width - 2*border:
        width_block = width - 2*border
    if height_block > height - 2*border:
        height_block = height - 2*border
    
    target_width = (width - 2*border)*2
    target_height = (height - 2*border)*2
    
    width_num = (width - 2*border)/width_block
    if (width - 2*border) > width_num * width_block:
        width_num += 1
    height_num = (height - 2*border)/height_block
    if (height - 2*border) > height_num * height_block:
        height_num += 1
        
    output = np.zeros((3, target_height, target_width), dtype=np.float)
    
    patch = list()
    region = list()
    
    for i in range(width_num):
        for j in range(height_num):
            x1 = i*width_block;
            x2 = min(x1+width_block+2*border, width)
            y1 = j*height_block;
            y2 = min(y1+height_block+2*border, height)
            patch.append(image[:,y1:y2,x1:x2])
            
            x1_ =2*i*width_block
            x2_ = min(x1_+2*width_block, target_width)
            y1_ = 2*j*height_block;
            y2_ = min(y1_+2*height_block, target_height)
            region.append([y1_,y2_,x1_,x2_])
    
    return [patch, region, output]


def main():

    net = caffe.Net('upconv_7.prototxt', net, caffe.TEST)

    im = Image.open(input_file)
    in2_ = np.array(im, dtype=np.float32)
    #im = im.resize((200,139), Image.BICUBIC)
    im = ImageOps.expand(im, border=7,fill='black')
    in_ = np.array(im, dtype=np.float32)
    in_ = in_ / 255.0
    in_ = in_.transpose((2,0,1))

    time1 = datetime.datetime.now()

    [patch, region, output] = getblock(in_, block_Width, block_height, 7)

    for idx in range(len(patch)):
        
        net.blobs['input'].reshape(1, *(patch[idx]).shape)
        net.blobs['input'].data[...] = (patch[idx])
        
        net.forward()
        output_idx = net.blobs['conv7'].data[0][:,:,:]
        
        output[:,region[idx][0]:region[idx][1],region[idx][2]:region[idx][3]] = output_idx

    output = np.minimum(output,1.0)
    output = np.maximum(output,0.0)
    output = np.rint(output*255.0)
    output = np.array(output, dtype=np.uint8)
        
    scipy.misc.imsave(output_file, output)


In [ ]:
image = Variable(ToTensor()(image), volatile=True).unsqueeze(0)
if TEST_MODE:
    image = image.cuda()

start = time.clock()
out = model(image)
elapsed = (time.clock() - start)
print('cost' + str(elapsed) + 's')
out_img = ToPILImage()(out[0].data.cpu())



In [ ]:
out_img.save('out_srf_' + str(UPSCALE_FACTOR) + '_' + IMAGE_NAME)